# Global Minimial Viable Products (1)

Combine:
* SMAP available porosity
* ERA5 runoff
* MERIT terrain slope
* USDA soil texture class

into global product for PRZ water restoration

In [203]:
import ee
import numpy as np

import sys
sys.path.insert(1, '../datasets/SMAP-USDA/')  # until we package this
from earthshot import water_viz as vis
from earthshot import water_common as common
from earthshot import normalize as norm
from earthshot import mon_stats
from statistics import mean
import folium
from folium import plugins
import geemap.eefolium as geemap

In [204]:
#ee.Authenticate()
ee.Initialize()

In [205]:
#import MAR swales features, local oregon scores, common parameters
bbox_global = ee.Geometry.BBox(-180,-90,180,90)
bbox_oregon = ee.Geometry.Polygon([[[-124.1, 43.38], [-124.1, 42.88], [-123.6, 42.88], [-123.6, 43.38]]])

MAR_swales = ee.FeatureCollection('users/amgadellaboudy/MAR_swales_collection')
oregon_scores = ee.Image('users/amgadellaboudy/oregon_addedScore').select('b1')

#print(common.bboxes())
print(norm.img_range(oregon_scores, area_of_interest = common.bboxes()['conus']))

[0.0008100000559352338, 0.9390730261802673]


In [206]:
Permiability_1 = ee.FeatureCollection('users/amgadellaboudy/Permiability_Data')
perma = 'Log[arithmetic mean or single values (Used Permiability)'
keepProperties = ee.List([perma])

def geom(sample):
    lat = ee.Number(sample.get('Latitude').getInfo())
    lon = ee.Number(sample.get('Longitude').getInfo())
    coords = ee.Geometry.Point(ee.List([lon, lat]))
    keepProperties = ee.List([perma])
    return ee.Feature(coords).copyProperties(sample, keepProperties)
    #perm_1 =  ee.Feature(ee.Geometry.Point([lon, lat]), {'perma': perm})
    #return perm_1

#perm_mapped = Permiability.map(geom)
#perm_flattened = perm_mapped.flatten()
#perm_img = perm_mapped.reduceToImage(['perma'], ee.Reducer.mean())
#img_mean = perm_img.reduceRegion(reducer = ee.Reducer.mean(), geometry = bbox_global, scale = 100)
#img_std = perm_img.reduceRegion(reducer = ee.Reducer.stdDev(), geometry = bbox_global, scale = 100)
img = Permiability_1.reduceToImage(['logK_Ice_x'], ee.Reducer.mean())

In [207]:
#perm_filtered = perm_mapped.filter(ee.Filter.notNull([perma]))
#combinedReducer = ee.Reducer.mean().combine(reducer2 = ee.Reducer.stdDev())
#mean = perm_mapped.reduceColumns(reducer= ee.Reducer.mean(), selectors= keepProperties)
#tdDev = perm_mapped.reduceColumns(reducer= ee.Reducer.stdDev(), selectors= keepProperties)

In [208]:
#interpolated =(perm_mapped.inverseDistance(range= 1000, propertyName= perma, mean= mean.get('mean'), 
  #              stdDev= stdDev.get('stdDev'), gamma = 0.3, reducer= ee.Reducer.mean()))
#print(norm.img_range(interpolated, area_of_interest= bbox_global))

In [209]:
band_viz = {'min': -1600, 'max': -1400, 'dimensions': 512,
  'palette': ['Red', 'Yellow', 'Blue']}

In [210]:
bbox_us = common.bboxes()['conus']

xx = (ee.batch.Export.image.toAsset(img, description= 'Image Export', 
                         assetId = 'users/amgadellaboudy/Permiability_Image',
                        scale = 1000000,
                        maxPixels = 10000000000000,
                        region = bbox_global)).start()

In [211]:
the_map = geemap.Map()

the_map.addLayer(img, band_viz, name = 'Outcrop Permiability')
vis.folium_display(the_map)

In [212]:
oregon_scores_scaled = (norm.img_scale(
    oregon_scores, area_of_interest= bbox_oregon))

# SMAP-USDA Available Porosity
More available porosity is better.

In [213]:
# Sum up available porosity in mm across the year
smap_usda_clim = ee.ImageCollection('users/jamesmcc/smap_usda_climatology')
avail_porosity = (smap_usda_clim
                  .filter(ee.Filter.eq('band', 'avail_porosity_mm')))

avail_porosity_img = avail_porosity.sum()

avail_porosity_img_scaled = (norm.img_scale(
    avail_porosity_img, area_of_interest= bbox_global))

## ERA5 Runoff

In [214]:
# Sum up runoff across the year to include in product score
runoff_clim = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY").select('surface_runoff')

runoff_clim_m = mon_stats.bands_avgs(['surface_runoff'], runoff_clim)

runoff_img = ee.ImageCollection(runoff_clim_m['avgs'].get('surface_runoff')).sum().multiply(720)
runoff_mask = runoff_img.gte(0.01) #values masked to areas that runoff 50 mm/ year at least
runoff_img_masked = runoff_img.updateMask(runoff_mask)

runoff_img_scaled = (norm.img_scale(
    runoff_img_masked, 
    area_of_interest= bbox_global))

## MERIT Terrain Slope

In [215]:
#include slope as a factor in scoring product, mask for lower slopes
slope_img = ee.Image('users/jamesmcc/merit_slope/merit_terrain_slope')
slope_mask = slope_img.gt(0).And(slope_img.lte(10))
slope_img_masked = slope_img.updateMask(slope_mask).pow(-1)
slope_img_scaled = norm.img_scale(slope_img_masked, area_of_interest = bbox_global)



# Soil Types

In [216]:
#incorporate soil types as a factor in scoring for PRZ
soil_types = ee.Image("OpenLandMap/SOL/SOL_TEXTURE-CLASS_USDA-TT_M/v02")

In [217]:
#categorizing soil types and depths based on retaining plant-available water, grouping top soils and bottom soils together
top_soils = [5,7,8,10]
medium_soils = [2,4,6,9]
low_soils = [1,3,11,12]
soil_0 = soil_types.expression(
        "(b('b0') == 12) ? 1.0" +
        ": (b('b0') == 11) ? 1.0" +
        ": (b('b0') == 10) ? 1.0" +
        ": (b('b0') == 9) ? 1.0" +
        ": (b('b0') == 8) ? 0.55" +
        ": (b('b0') == 7) ? 0.55" +
        ": (b('b0') == 6) ? 0.55" +
        ": (b('b0') == 5) ? 0.55" +
        ": (b('b0') == 4) ? 0.1" +
        ": (b('b0') == 3) ? 0.1" +
        ": (b('b0') == 2) ? 0.1" +
        ": (b('b0') == 1) ? 0.1" +
        ": 0")

soil_10 = soil_types.expression(
        "(b('b10') == 12) ? 1.0" +
        ": (b('b10') == 11) ? 1.0" +
        ": (b('b10') == 10) ? 1.0" +
        ": (b('b10') == 9) ? 1.0" +
        ": (b('b10') == 8) ? 0.55" +
        ": (b('b10') == 7) ? 0.55" +
        ": (b('b10') == 6) ? 0.55" +
        ": (b('b10') == 5) ? 0.55" +
        ": (b('b10') == 4) ? 0.1" +
        ": (b('b10') == 3) ? 0.1" +
        ": (b('b10') == 2) ? 0.1" +
        ": (b('b10') == 1) ? 0.1" +
        ": 0")

soil_30 = soil_types.expression(
        "(b('b30') == 12) ? 1.0" +
        ": (b('b30') == 11) ? 1.0" +
        ": (b('b30') == 10) ? 1.0" +
        ": (b('b30') == 9) ? 1.0" +
        ": (b('b30') == 8) ? 0.55" +
        ": (b('b30') == 7) ? 0.55" +
        ": (b('b30') == 6) ? 0.55" +
        ": (b('b30') == 5) ? 0.55" +
        ": (b('b30') == 4) ? 0.1" +
        ": (b('b30') == 3) ? 0.1" +
        ": (b('b30') == 2) ? 0.1" +
        ": (b('b30') == 1) ? 0.1" +
        ": 0")

soil_60 = soil_types.expression(
        "(b('b60') == 5) ? 1.0" +
        ": (b('b60') == 7) ? 1.0" +
        ": (b('b60') == 8) ? 1.0" +
        ": (b('b60') == 10) ? 1.0" +
        ": (b('b60') == 2) ? 0.55" +
        ": (b('b60') == 4) ? 0.55" +
        ": (b('b60') == 6) ? 0.55" +
        ": (b('b60') == 9) ? 0.55" +
        ": (b('b60') == 1) ? 0.1" +
        ": (b('b60') == 3) ? 0.1" +
        ": (b('b60') == 11) ? 0.1" +
        ": (b('b60') == 12) ? 0.1" +
        ": 0")

soil_100 = soil_types.expression(
        "(b('b100') == 5) ? 1.0" +
        ": (b('b100') == 7) ? 1.0" +
        ": (b('b100') == 8) ? 1.0" +
        ": (b('b100') == 10) ? 1.0" +
        ": (b('b100') == 2) ? 0.55" +
        ": (b('b100') == 4) ? 0.55" +
        ": (b('b100') == 6) ? 0.55" +
        ": (b('b100') == 9) ? 0.55" +
        ": (b('b100') == 1) ? 0.1" +
        ": (b('b100') == 3) ? 0.1" +
        ": (b('b100') == 11) ? 0.1" +
        ": (b('b100') == 12) ? 0.1" +
        ": 0")

soil_200 = soil_types.expression(
        "(b('b200') == 5) ? 1.0" +
        ": (b('b200') == 7) ? 1.0" +
        ": (b('b200') == 8) ? 1.0" +
        ": (b('b200') == 10) ? 1.0" +
        ": (b('b200') == 2) ? 0.55" +
        ": (b('b200') == 4) ? 0.55" +
        ": (b('b200') == 6) ? 0.55" +
        ": (b('b200') == 9) ? 0.55" +
        ": (b('b200') == 1) ? 0.1" +
        ": (b('b200') == 3) ? 0.1" +
        ": (b('b200') == 11) ? 0.1" +
        ": (b('b200') == 12) ? 0.1" +
        ": 0")

top_soils = soil_0.expression('soil_0 + soil_10 + soil_30',
                             {'soil_0': soil_0.select('constant'),
                             'soil_10': soil_10.select('constant'),
                             'soil_30': soil_30.select('constant')})

bottom_soils = soil_60.expression('soil_60 + soil_100 + soil_200',
                                 {'soil_60': soil_60.select('constant'),
                                  'soil_100': soil_100.select('constant'),
                                  'soil_200': soil_200.select('constant')})

In [218]:
#scaling top soils and bottom soils
top_soils_scaled = norm.img_scale(top_soils, area_of_interest= bbox_global)
bottom_soils_scaled = norm.img_scale(bottom_soils, area_of_interest= bbox_global)

In [219]:
#incorporate organic matter as a factor in scoring for PRZ
organic_matter = ee.Image("OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02").reduce(ee.Reducer.sum())
organic_matter_scaled = norm.img_scale(organic_matter, area_of_interest= bbox_global)

In [220]:
print(norm.img_range(organic_matter_scaled, area_of_interest = bbox_global))

[0, 1]


## PRZ Score
Simply give higher scores to places that have both high runoff, available porosity, favorable soil type and slope. Masked for lower slopes and higher runoffs.

In [221]:
score = (runoff_img_scaled.multiply(.2)
         .add(avail_porosity_img_scaled.multiply(.2))
         .add(slope_img_scaled.multiply(.2))
         .add(organic_matter_scaled.multiply(.2))
         .add(top_soils_scaled.multiply(.1))
        .add(bottom_soils_scaled.multiply(.1)))

score_scaled = norm.img_scale(score, area_of_interest= bbox_global)

In [222]:
#visualizing PRZ score, other variables
import branca

score_range = norm.img_range(score_scaled, area_of_interest = bbox_global)
palette_name = 'RdYlBu'
palette_len = 11
palette = vis.brewer[palette_name][palette_len][::-1]

colormap = branca.colormap.LinearColormap(colors = ['Red', 'Yellow', 'Blue'], vmin = 0, vmax =1)
colormap = colormap.to_step(n=10)
colormap.caption = 'PRZ Score'

box_corners = bbox_global.toGeoJSON()['coordinates'][0]
center_lon = mean([corner[0] for corner in box_corners])
center_lat = mean([corner[1] for corner in box_corners])

vis_params = {
    'min': score_range[0], 'max': score_range[1], 'dimensions': 512,
    'palette': ['Red', 'Yellow', 'Blue']}


In [223]:
#creating the map, visualizing score


the_map = vis.folium_map(location=[center_lat, center_lon], zoom_start=1, height=400)


the_map.add_ee_layer(score_scaled, vis_params, name = 'PRZ Score')
#the_map.add_ee_layer(MAR_swales.draw(color= '006600', pointRadius = 5, strokeWidth= 5), {}, name = 'MAR Swale locations')
colormap.add_to(the_map)


#the_map.add_ee_layer(oregon_scores, vis_params, name = 'Oregon Scores')
#avail_porosity_masked = avail_porosity_img.updateMask(runoff_mask).updateMask(slope_mask)
#the_map.add_ee_layer(score_2, vis_params, name = 'PRZ Score-2')
#the_map.add_ee_layer(top_soils_scaled, vis_params, name = 'Top Soils')
#the_map.add_ee_layer(avail_porosity_masked, vis_params_poros, name = 'Available Porosity')
#the_map.add_ee_layer(runoff_img_masked, vis_params_runoff, name = 'Runoff')
#the_map.add_ee_layer(slope_img_masked, vis_params_slope, name = 'Slope')
#the_map.add_ee_layer(slope_img_scaled, vis_params, name = 'Slope')
#the_map.add_ee_layer(top_soils_scaled, vis_params, name = 'Top Soils')

vis.folium_display(the_map)



In [224]:
#Export map to GEE

#collection_name = (
#    'users/amgadellaboudy/Global_PRZ_Score')
#score_asset = ee.data.createAsset({'type': 'ImageCollection'}, collection_name)

#xx = ee.batch.Export.image.toAsset(
 #   score, 
 #   assetId = score_asset['id'] + '_collection', 
 #   region = bbox_global,
  #  maxPixels = 100000000000).start()

In [225]:
#xx1 = ee.batch.Export.image.toDrive(
  # runoff_img_scaled, 
  #  folder = 'Desktop/',
   # region = common.bboxes()['conus'],
   # maxPixels = 100000000000).start()

In [226]:
#xx2= ee.batch.Export.image.toDrive(
  #  slope_mask, 
   # description = 'slope',
   # region = common.bboxes()['conus'],
   # maxPixels = 100000000000).start()

In [227]:
#xx3= ee.batch.Export.image.toDrive(
  #  top_soils_scaled, 
  #  description = 'top soils',
  #  region = common.bboxes()['conus'],
  #  maxPixels = 100000000000).start()

#xx4= ee.batch.Export.image.toDrive(
  #  bottom_soils_scaled, 
  #  description = 'bottom soils',
   # region = common.bboxes()['conus'],
   # maxPixels = 100000000000).start()